In [0]:
# Join
# Question 1 - How can you produce a list of the start times for bookings by members named 'David Farrell'?

bookings_df = spark.table("bookings")
members_df = spark.table("members")

# Join
result_df = bookings_df.join(members_df, bookings_df.memid == members_df.memid)

# Filter
result_df = result_df.filter((members_df.firstname == "David") & (members_df.surname == "Farrell"))

# Select 
start_times_df = result_df.select("starttime")

# Display 
start_times_df.show()

+-------------------+
|          starttime|
+-------------------+
|2012-09-18 09:00:00|
|2012-09-18 17:30:00|
|2012-09-18 13:30:00|
|2012-09-18 20:00:00|
|2012-09-19 09:30:00|
|2012-09-19 15:00:00|
|2012-09-19 12:00:00|
|2012-09-20 15:30:00|
|2012-09-20 11:30:00|
|2012-09-20 14:00:00|
|2012-09-21 10:30:00|
|2012-09-21 14:00:00|
|2012-09-22 08:30:00|
|2012-09-22 17:00:00|
|2012-09-23 08:30:00|
|2012-09-23 17:30:00|
|2012-09-23 19:00:00|
|2012-09-24 08:00:00|
|2012-09-24 16:30:00|
|2012-09-24 12:30:00|
+-------------------+
only showing top 20 rows



In [0]:
# Join
# Question 2 - How can you produce a list of the start times for bookings for tennis courts, for the date '2012-09-21'? Return a list of start time and facility name pairings, ordered by the time.
from pyspark.sql.functions import col, to_date, date_format

bookings_df = spark.table("bookings")
facilities_df = spark.table("facilities")

# Join
result_df = bookings_df.join(facilities_df, bookings_df.facid == facilities_df.facid)

# Filter
result_df = result_df.filter(
    (facilities_df.name.contains("Tennis Court")) & 
    (date_format(col("starttime"), "yyyy-MM-dd") == "2012-09-21")
)

# Select
result_df = result_df.select("starttime", "name")

# Order
result_df = result_df.orderBy("starttime")

# Display
result_df.show()



+-------------------+--------------+
|          starttime|          name|
+-------------------+--------------+
|2012-09-21 08:00:00|Tennis Court 1|
|2012-09-21 08:00:00|Tennis Court 2|
|2012-09-21 09:30:00|Tennis Court 1|
|2012-09-21 10:00:00|Tennis Court 2|
|2012-09-21 11:30:00|Tennis Court 2|
|2012-09-21 12:00:00|Tennis Court 1|
|2012-09-21 13:30:00|Tennis Court 1|
|2012-09-21 14:00:00|Tennis Court 2|
|2012-09-21 15:30:00|Tennis Court 1|
|2012-09-21 16:00:00|Tennis Court 2|
|2012-09-21 17:00:00|Tennis Court 1|
|2012-09-21 18:00:00|Tennis Court 2|
+-------------------+--------------+



In [0]:
# Join
# Question 3 - How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).
from pyspark.sql.functions import col

members_df = spark.table("members")

# Join
result_df = members_df.alias("mems").join(
    members_df.alias("recs"),
    col("recs.memid") == col("mems.recommendedby"),
    how="left"
)

# Select and Order
result_df = result_df.select(
    col("mems.firstname").alias("memfname"),
    col("mems.surname").alias("memsname"),
    col("recs.firstname").alias("recfname"),
    col("recs.surname").alias("recsname")
).orderBy("memsname", "memfname")

# Display
result_df.show()

+---------+---------+---------+--------+
| memfname| memsname| recfname|recsname|
+---------+---------+---------+--------+
| Florence|    Bader|   Ponder|Stibbons|
|     Anne|    Baker|   Ponder|Stibbons|
|  Timothy|    Baker|   Jemima| Farrell|
|      Tim|   Boothe|      Tim|  Rownam|
|   Gerald|  Butters|   Darren|   Smith|
|     Joan|   Coplin|  Timothy|   Baker|
|    Erica|  Crumpet|    Tracy|   Smith|
|    Nancy|     Dare|   Janice|Joplette|
|    David|  Farrell|     null|    null|
|   Jemima|  Farrell|     null|    null|
|    GUEST|    GUEST|     null|    null|
|  Matthew|  Genting|   Gerald| Butters|
|     John|     Hunt|Millicent| Purview|
|    David|    Jones|   Janice|Joplette|
|  Douglas|    Jones|    David|   Jones|
|   Janice| Joplette|   Darren|   Smith|
|     Anna|Mackenzie|   Darren|   Smith|
|  Charles|     Owen|   Darren|   Smith|
|    David|   Pinker|   Jemima| Farrell|
|Millicent|  Purview|    Tracy|   Smith|
+---------+---------+---------+--------+
only showing top

In [0]:
# Join
# Question 4 - How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.
from pyspark.sql import functions as F

members_df = spark.table("members")
bookings_df = spark.table("bookings")
facilities_df = spark.table("facilities")

# Join 
result_df = members_df.join(bookings_df, members_df.memid == bookings_df.memid)
result_df = result_df.join(facilities_df, bookings_df.facid == facilities_df.facid)

# Filter
result_df = result_df.filter(facilities_df.name.isin("Tennis Court 2", "Tennis Court 1"))

# Select 
result_df = result_df.select(
    F.concat(members_df.firstname, F.lit(" "), members_df.surname).alias("member"),
    facilities_df.name.alias("facility")
)

# Remove duplicates
result_df = result_df.distinct()

# Order
result_df = result_df.orderBy("member", "facility")

# Display 
result_df.show()

+--------------+--------------+
|        member|      facility|
+--------------+--------------+
|    Anne Baker|Tennis Court 1|
|    Anne Baker|Tennis Court 2|
|  Burton Tracy|Tennis Court 1|
|  Burton Tracy|Tennis Court 2|
|  Charles Owen|Tennis Court 1|
|  Charles Owen|Tennis Court 2|
|  Darren Smith|Tennis Court 2|
| David Farrell|Tennis Court 1|
| David Farrell|Tennis Court 2|
|   David Jones|Tennis Court 1|
|   David Jones|Tennis Court 2|
|  David Pinker|Tennis Court 1|
| Douglas Jones|Tennis Court 1|
| Erica Crumpet|Tennis Court 1|
|Florence Bader|Tennis Court 1|
|Florence Bader|Tennis Court 2|
|   GUEST GUEST|Tennis Court 1|
|   GUEST GUEST|Tennis Court 2|
|Gerald Butters|Tennis Court 1|
|Gerald Butters|Tennis Court 2|
+--------------+--------------+
only showing top 20 rows



In [0]:
# Join
# Question 5 - How can you output a list of all members, including the individual who recommended them (if any), without using any joins? Ensure that there are no duplicates in the list, and that each firstname + surname pairing is formatted as a column and ordered.
from pyspark.sql import functions as F

members_df = spark.table("members")

# Column
members_with_full_name = members_df.withColumn(
    "member", 
    F.concat(F.col("firstname"), F.lit(" "), F.col("surname"))
)

# Subquery
result_df = members_with_full_name.withColumn(
    "recommender",
    F.expr("""
        (SELECT CONCAT(recs.firstname, ' ', recs.surname)
         FROM members recs
         WHERE recs.memid = recommendedby)
    """)
)

# Select 
final_df = (result_df
    .select("member", "recommender")
    .distinct()
    .orderBy("member")
)

# Display 
final_df.show()

+--------------------+-----------+
|              member|recommender|
+--------------------+-----------+
|      Anna Mackenzie|       null|
|          Anne Baker|       null|
|        Burton Tracy|       null|
|        Charles Owen|       null|
|        Darren Smith|       null|
|       David Farrell|       null|
|         David Jones|       null|
|        David Pinker|       null|
|       Douglas Jones|       null|
|       Erica Crumpet|       null|
|      Florence Bader|       null|
|         GUEST GUEST|       null|
|      Gerald Butters|       null|
|    Henrietta Rumney|       null|
|Henry Worthington...|       null|
| Hyacinth Tupperware|       null|
|          Jack Smith|       null|
|     Janice Joplette|       null|
|      Jemima Farrell|       null|
|         Joan Coplin|       null|
+--------------------+-----------+
only showing top 20 rows



In [0]:
# Aggregation
# Question 1 - Produce a count of the number of recommendations each member has made. Order by member ID.
from pyspark.sql.functions import col, count

members_df = spark.table("members")

# Filter
filtered_df = members_df.filter(col("recommendedby").isNotNull())

# Group by 
grouped_df = filtered_df.groupBy("recommendedby").agg(count("*").alias("recommendation_count"))

# Order by
result_df = grouped_df.orderBy("recommendedby")

# Display 
result_df.show()

+-------------+--------------------+
|recommendedby|recommendation_count|
+-------------+--------------------+
|            1|                   5|
|            2|                   3|
|            3|                   1|
|            4|                   2|
|            5|                   1|
|            6|                   1|
|            9|                   2|
|           11|                   1|
|           13|                   2|
|           15|                   1|
|           16|                   1|
|           20|                   1|
|           30|                   1|
+-------------+--------------------+



In [0]:
# Aggregation
# Question 2 - Produce a list of the total number of slots booked per facility. For now, just produce an output table consisting of facility id and slots, sorted by facility id.
from pyspark.sql.functions import sum

bookings_df = spark.table("bookings")

# Group by 
grouped_df = bookings_df.groupBy("facid").agg(sum("slots").alias("Total Slots"))

# Order by
result_df = grouped_df.orderBy("facid")

# Display 
result_df.show()

+-----+-----------+
|facid|Total Slots|
+-----+-----------+
|    0|       1320|
|    1|       1278|
|    2|       1209|
|    3|        830|
|    4|       1404|
|    5|        228|
|    6|       1104|
|    7|        908|
|    8|        911|
+-----+-----------+



In [0]:
# Aggregation
# Question 3 - Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.
from pyspark.sql.functions import col, sum, date_format

bookings_df = spark.table("bookings")

# Filter 
filtered_df = bookings_df.filter(
    (col("starttime") >= "2012-09-01") & (col("starttime") < "2012-10-01")
)

# Group by 
grouped_df = filtered_df.groupBy("facid").agg(sum("slots").alias("Total Slots"))

# Order by
result_df = grouped_df.orderBy("Total Slots")

# Display 
result_df.show()

+-----+-----------+
|facid|Total Slots|
+-----+-----------+
|    5|        122|
|    3|        422|
|    7|        426|
|    8|        471|
|    6|        540|
|    2|        570|
|    1|        588|
|    0|        591|
|    4|        648|
+-----+-----------+



In [0]:
# Aggregation
# Question 4 - Produce a list of the total number of slots booked per facility per month in the year of 2012. Produce an output table consisting of facility id and slots, sorted by the id and month.
from pyspark.sql.functions import col, sum, month, year

bookings_df = spark.table("bookings")

# Filter 
filtered_df = bookings_df.filter(year(col("starttime")) == 2012)

# Group by
grouped_df = filtered_df.withColumn("month", month(col("starttime"))) \
    .groupBy("facid", "month") \
    .agg(sum("slots").alias("Total Slots"))

# Order by
result_df = grouped_df.orderBy("facid", "month")

# Display 
result_df.show()

+-----+-----+-----------+
|facid|month|Total Slots|
+-----+-----+-----------+
|    0|    7|        270|
|    0|    8|        459|
|    0|    9|        591|
|    1|    7|        207|
|    1|    8|        483|
|    1|    9|        588|
|    2|    7|        180|
|    2|    8|        459|
|    2|    9|        570|
|    3|    7|        104|
|    3|    8|        304|
|    3|    9|        422|
|    4|    7|        264|
|    4|    8|        492|
|    4|    9|        648|
|    5|    7|         24|
|    5|    8|         82|
|    5|    9|        122|
|    6|    7|        164|
|    6|    8|        400|
+-----+-----+-----------+
only showing top 20 rows



In [0]:
# Aggregation
# Question 5 - Find the total number of members (including guests) who have made at least one booking.

bookings_df = spark.table("bookings")

# Select 
distinct_members_count = bookings_df.select("memid").distinct().count()

# Display
print(f"Total number of members (including guests) who have made at least one booking: {distinct_members_count}")


Total number of members (including guests) who have made at least one booking: 30


In [0]:
# Aggregation
# Question 6 - Produce a list of each member name, id, and their first booking after September 1st 2012. Order by member ID.
from pyspark.sql.functions import col, min

bookings_df = spark.table("bookings")
members_df = spark.table("members")

# Join
joined_df = bookings_df.alias("bks").join(members_df.alias("mems"), col("bks.memid") == col("mems.memid"))

# Filter 
filtered_df = joined_df.filter(col("bks.starttime") >= "2012-09-01")

# Group by 
grouped_df = filtered_df.groupBy(col("mems.memid"), col("mems.surname"), col("mems.firstname")).agg(min(col("bks.starttime")).alias("first_booking"))

# Order by
result_df = grouped_df.orderBy(col("mems.memid"))

# Select 
result_df = result_df.select(col("mems.surname"), col("mems.firstname"), col("mems.memid"), col("first_booking"))

# Display 
result_df.show()


+---------+---------+-----+-------------------+
|  surname|firstname|memid|      first_booking|
+---------+---------+-----+-------------------+
|    GUEST|    GUEST|    0|2012-09-01 08:00:00|
|    Smith|   Darren|    1|2012-09-01 09:00:00|
|    Smith|    Tracy|    2|2012-09-01 11:30:00|
|   Rownam|      Tim|    3|2012-09-01 16:00:00|
| Joplette|   Janice|    4|2012-09-01 15:00:00|
|  Butters|   Gerald|    5|2012-09-02 12:30:00|
|    Tracy|   Burton|    6|2012-09-01 15:00:00|
|     Dare|    Nancy|    7|2012-09-01 12:30:00|
|   Boothe|      Tim|    8|2012-09-01 08:30:00|
| Stibbons|   Ponder|    9|2012-09-01 11:00:00|
|     Owen|  Charles|   10|2012-09-01 11:00:00|
|    Jones|    David|   11|2012-09-01 09:30:00|
|    Baker|     Anne|   12|2012-09-01 14:30:00|
|  Farrell|   Jemima|   13|2012-09-01 09:30:00|
|    Smith|     Jack|   14|2012-09-01 11:00:00|
|    Bader| Florence|   15|2012-09-01 10:30:00|
|    Baker|  Timothy|   16|2012-09-01 15:00:00|
|   Pinker|    David|   17|2012-09-01 08

In [0]:
# String & Date
# Question 1 - Output the names of all members, formatted as 'Surname, Firstname'
from pyspark.sql.functions import concat_ws

members_df = spark.table("members")

# Concatenate 
result_df = members_df.select(concat_ws(", ", col("surname"), col("firstname")).alias("name"))

# Display 
result_df.show(truncate=False)

+----------------+
|name            |
+----------------+
|GUEST, GUEST    |
|Smith, Darren   |
|Smith, Tracy    |
|Rownam, Tim     |
|Joplette, Janice|
|Butters, Gerald |
|Tracy, Burton   |
|Dare, Nancy     |
|Boothe, Tim     |
|Stibbons, Ponder|
|Owen, Charles   |
|Jones, David    |
|Baker, Anne     |
|Farrell, Jemima |
|Smith, Jack     |
|Bader, Florence |
|Baker, Timothy  |
|Pinker, David   |
|Genting, Matthew|
|Mackenzie, Anna |
+----------------+
only showing top 20 rows



In [0]:
# String & Date
# Question 2 - Perform a case-insensitive search to find all facilities whose name begins with 'tennis'. Retrieve all columns.
from pyspark.sql.functions import col, upper

facilities_df = spark.table("facilities")

# Filter
result_df = facilities_df.filter(upper(col("name")).like("TENNIS%"))

# Display 
result_df.show(truncate=False)

+-----+--------------+----------+---------+-------------+------------------+
|facid|name          |membercost|guestcost|initialoutlay|monthlymaintenance|
+-----+--------------+----------+---------+-------------+------------------+
|0    |Tennis Court 1|5.0       |25.0     |10000        |200               |
|1    |Tennis Court 2|5.0       |25.0     |8000         |200               |
+-----+--------------+----------+---------+-------------+------------------+



In [0]:
# String & Date
# Question 3 - You've noticed that the club's member table has telephone numbers with very inconsistent formatting. You'd like to find all the telephone numbers that contain parentheses, returning the member ID and telephone number sorted by member ID.
from pyspark.sql.functions import col

members_df = spark.table("members")

# Filter
result_df = members_df.filter(col("telephone").rlike(r"[()]"))

# Select 
result_df = result_df.select("memid", "telephone").orderBy("memid")

# Display 
result_df.show(truncate=False)

+-----+--------------+
|memid|telephone     |
+-----+--------------+
|0    |(000) 000-0000|
|3    |(844) 693-0723|
|4    |(833) 942-4710|
|5    |(844) 078-4130|
|6    |(822) 354-9973|
|7    |(833) 776-4001|
|8    |(811) 433-2547|
|9    |(833) 160-3900|
|10   |(855) 542-5251|
|11   |(844) 536-8036|
|13   |(855) 016-0163|
|14   |(822) 163-3254|
|15   |(833) 499-3527|
|20   |(811) 972-1377|
|21   |(822) 661-2898|
|22   |(822) 499-2232|
|24   |(822) 413-1470|
|27   |(822) 989-8876|
|28   |(855) 755-9876|
|29   |(855) 894-3758|
+-----+--------------+
only showing top 20 rows



In [0]:
# String & Date
# Question 4 - You'd like to produce a count of how many members you have whose surname starts with each letter of the alphabet. Sort by the letter, and don't worry about printing out a letter if the count is 0.
from pyspark.sql.functions import col, count, substring

members_df = spark.table("members")

# Fiilter
members_with_letter_df = members_df.withColumn("letter", substring(col("surname"), 1, 1))

# Group by 
result_df = members_with_letter_df.groupBy("letter").agg(count("*").alias("count"))

# Order by 
result_df = result_df.orderBy("letter")

# Display the result
result_df.show()

+------+-----+
|letter|count|
+------+-----+
|     B|    5|
|     C|    2|
|     D|    1|
|     F|    2|
|     G|    2|
|     H|    1|
|     J|    3|
|     M|    1|
|     O|    1|
|     P|    2|
|     R|    2|
|     S|    6|
|     T|    2|
|     W|    1|
+------+-----+



In [0]:
# String & Date
# Question 5 - Produce a list of all the dates in October 2012. They can be output as a timestamp (with time set to midnight) or a date.
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType
from datetime import datetime, timedelta

start_date = datetime(2012, 10, 1)
end_date = datetime(2012, 10, 31)
date_list = [(start_date + timedelta(days=x),) for x in range((end_date - start_date).days + 1)]

date_df = spark.createDataFrame(date_list, ["ts"])

date_df = date_df.withColumn("ts", F.col("ts").cast(TimestampType()))

date_df = date_df.orderBy("ts")

date_df.show(31, truncate=False)


+-------------------+
|ts                 |
+-------------------+
|2012-10-01 00:00:00|
|2012-10-02 00:00:00|
|2012-10-03 00:00:00|
|2012-10-04 00:00:00|
|2012-10-05 00:00:00|
|2012-10-06 00:00:00|
|2012-10-07 00:00:00|
|2012-10-08 00:00:00|
|2012-10-09 00:00:00|
|2012-10-10 00:00:00|
|2012-10-11 00:00:00|
|2012-10-12 00:00:00|
|2012-10-13 00:00:00|
|2012-10-14 00:00:00|
|2012-10-15 00:00:00|
|2012-10-16 00:00:00|
|2012-10-17 00:00:00|
|2012-10-18 00:00:00|
|2012-10-19 00:00:00|
|2012-10-20 00:00:00|
|2012-10-21 00:00:00|
|2012-10-22 00:00:00|
|2012-10-23 00:00:00|
|2012-10-24 00:00:00|
|2012-10-25 00:00:00|
|2012-10-26 00:00:00|
|2012-10-27 00:00:00|
|2012-10-28 00:00:00|
|2012-10-29 00:00:00|
|2012-10-30 00:00:00|
|2012-10-31 00:00:00|
+-------------------+



In [0]:
# String & Date
# Question 6 - Return a count of bookings for each month, sorted by month
from pyspark.sql.functions import date_trunc, count, col

bookings_df = spark.table("bookings")

# Truncate 
truncated_month_df = bookings_df.withColumn("month", date_trunc("month", col("starttime")))

# Group by 
monthly_booking_counts_df = truncated_month_df.groupBy("month").agg(count("*").alias("booking_count"))

# Order by
sorted_monthly_booking_counts_df = monthly_booking_counts_df.orderBy("month")

# Display 
sorted_monthly_booking_counts_df.show(truncate=False)

+-------------------+-------------+
|month              |booking_count|
+-------------------+-------------+
|2012-07-01 00:00:00|658          |
|2012-08-01 00:00:00|1472         |
|2012-09-01 00:00:00|1913         |
|2013-01-01 00:00:00|1            |
+-------------------+-------------+

